In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')
#foldername should be changed to the name of the directory containing the file.
foldername = '/Longform-Question-Generation/'
%cd /content/drive/My\ Drive/$foldername

Mounted at /content/drive
/content/drive/My Drive/Longform-Question-Generation


In [2]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install openai
!pip install evaluate
!pip install rouge_score
!pip install bert_score

import openai
import os
import requests
import evaluate
import requests
import pickle
import torch
import pandas as pd
import numpy as np
from getpass import getpass
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from tqdm import tqdm
from datasets import load_from_disk, load_dataset, Dataset, load_metric,DatasetDict

from transformers.pipelines.pt_utils import KeyDataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=90fce930ec91a3de941cd407688a9b91f70cdbaa14c4c22db8efc5d44e76b21e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00


# Extra Cleaning

In [ ]:
#asks_ds=load_from_disk('./data/asks_cleaned_data')
#asks_ds_yake_top3=load_from_disk('./data/yake_top3_asks_cleaned_data')
#
#asks_ds_filtered=DatasetDict()
#
#for key in asks_ds:
#    asks_ds_filtered[key]=asks_ds[key].filter(lambda example: 'ask me anything' not in example['title'] 
#                             and 'megathread' not in example['title']
#                             and 'ama series' not in example['title'] )
#    
#yake_asks_ds_filtered=DatasetDict()
#for key in asks_ds_yake_top3:
#    yake_asks_ds_filtered[key]=asks_ds_yake_top3[key].filter(lambda example: 'ask me anything' not in example['title'] 
#                             and 'megathread' not in example['title']
#                             and 'ama series' not in example['title'] )
#
#asks_ds_filtered.save_to_disk('./data/asks_filtered_data')
#yake_asks_ds_filtered.save_to_disk('./data/asks_yaketop3_filtered_data')

In [15]:
model_list=['tinier_bart','bart-tiny-random','bart-small','bart-base','bart-large',
        'pegasus-arxiv','flan-t5-small','flan-t5-base']

model_list_large=['bart-large','pegasus-arxiv','flan-t5-base']

In [ ]:
asks_ds_filtered=load_from_disk('./data/asks_filtered_data')
yake_asks_ds_filtered=load_from_disk('./data/asks_yaketop3_filtered_data')

In [ ]:
asks_ds_unfiltered=load_from_disk('./data/asks_cleaned_data')
yake_asks_ds_unfiltered=load_from_disk('./data/yake_top3_asks_cleaned_data')

In [ ]:
(len(asks_ds_unfiltered['train'])-len(asks_ds_filtered['train']))/len(asks_ds_unfiltered['train'])*100

0.0359072157544904

In [ ]:
(len(asks_ds_unfiltered['validation'])-len(asks_ds_filtered['validation']))/len(asks_ds_unfiltered['validation'])*100

2.3300970873786406

# Pipelines of trained models


In [ ]:
max_length=50
batch_size=4
for model_name in model_list_large:
    if 'flan' in model_name.lower():
        continue

    print(f'working on model: {model_name}')

    
    
    if 'pegasus' not in model_name:
        checkpoint = f'dhmeltzer/{model_name}_askscience-qg'
    else:
        checkpoint = f'dhmeltzer/{model_name}-askscience-qg'

    model=pipeline("text2text-generation", 
               model=checkpoint,
               max_length=max_length,
               device=0,
               batch_size=batch_size,
               truncation=True)
    
    table=[]
    for i in tqdm(range(0,2012,4)):
        
        titles=asks_ds_filtered['validation'][i:i+batch_size]['title']
        answers=asks_ds_filtered['validation'][i:i+batch_size]['answers.text']
        
        if 'flan' in model_name.lower():
            answers=list(map(lambda x: 'generate a question: '+x,answers))
        #answers=list(map(lambda x:x[:512],answers))

        predictions=model(answers)
        predictions=list(map(lambda x:x['generated_text'],predictions))
        
        for (title,answer,prediction) in zip(titles,answers,predictions):
            table.append([answer,title,prediction])
        
    dataframe=pd.DataFrame(table,columns=['answer','title','prediction'])
    dataframe.to_csv(f'./results/{model_name}_validation_{max_length}')

working on model: bart-large


  2%|▏         | 10/503 [00:05<04:57,  1.66it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 503/503 [04:09<00:00,  2.02it/s]


working on model: pegasus-arxiv


  2%|▏         | 10/503 [00:12<10:55,  1.33s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 503/503 [13:21<00:00,  1.59s/it]


In [ ]:
max_length=50
batch_size=4
for model_name in model_list:
    #if 'flan' not in model_name.lower():
    #    continue
    print(f'working on model: {model_name}')

    checkpoint = f'dhmeltzer/{model_name}_yake_top3_asks_qg'

    model=pipeline("text2text-generation", 
               model=checkpoint,
               max_length=max_length,
               device=0,
               batch_size=batch_size,
               truncation=True)
    
    
    table=[]
    for i in tqdm(range(0,2012,4)):
        
        titles=yake_asks_ds_filtered['validation'][i:i+batch_size]['title']
        answers=yake_asks_ds_filtered['validation'][i:i+batch_size]['answers.text']

        if 'flan' in model_name.lower():
            answers=list(map(lambda x: 'generate a question: '+x,answers))

        answers=list(map(lambda x:x[:512],answers))

        predictions=model(answers)
        predictions=list(map(lambda x:x['generated_text'],predictions))
        
        for (title,answer,prediction) in zip(titles,answers,predictions):
            table.append([answer,title,prediction])

    
    dataframe=pd.DataFrame(table,columns=['answer','title','prediction'])
    dataframe.to_csv(f'./results/yake_top3_{model_name}_validation_{max_length}')

In [ ]:
df_original=pd.DataFrame([],columns=['answer','title'])
max_length=50

df_original['answer']=asks_ds_filtered['validation']['answers.text']
df_original['title']=asks_ds_filtered['validation']['title']

model_list_large=['bart-large','pegasus-arxiv','flan-t5-base']

for model_name in model_list_large:
    df_file=f'./results/{model_name}_validation_{max_length}'
    df_original[model_name]=pd.read_csv(df_file)['prediction']

df_original.to_csv('results/large_model_results')

In [ ]:
df_augmented=pd.DataFrame([],columns=['answer','title'])
max_length=50

df_augmented['answer']=yake_asks_ds_filtered['validation']['answers.text']
df_augmented['title']=yake_asks_ds_filtered['validation']['title']

model_list_large=['bart-large','pegasus-arxiv','flan-t5-base']

for model_name in model_list_large:
    df_file=f'./results/yake_top3_{model_name}_validation_{max_length}'
    df_augmented[model_name]=pd.read_csv(df_file)['prediction']
df_augmented.to_csv('results/augmented_large_model_results')

# Zero-shot Generation

In [ ]:
rouge=evaluate.load('rouge')
bertscore=evaluate.load('bertscore')

## FLAN-T5-XXL

In [ ]:
secret_hf = getpass('Enter your Huggingface key: ')

API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-xxl"
headers = {"Authorization": f"Bearer {secret_hf}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

Enter your Huggingface key: ··········


In [ ]:
asks_ds_filtered=load_from_disk('./data/asks_filtered_data')
asks_ds_yake_top3_filtered=load_from_disk('./data/asks_yaketop3_filtered_data')

In [ ]:
flanT5_tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-xxl')

In [ ]:
input=flanT5_tokenizer.encode('this is a test hello there old friend')
flanT5_tokenizer.decode(input,skip_special_tokens=True)

'this is a test hello there old friend'

In [ ]:
flan_t5_xxl_preds = []
i=0
while len(flan_t5_xxl_preds)<len(asks_ds_filtered['validation']['answers.text']):
    i=len(flan_t5_xxl_preds)

    answer=asks_ds_filtered['validation'][i]['answers.text']
    prompt=f"generate a question: {answer}"
    
    prompt_ids=flanT5_tokenizer.encode(prompt)[:512]
    prompt=flanT5_tokenizer.decode(prompt_ids,skip_special_tokens=True)

    question = query({'inputs':prompt})
    question = question[0]['generated_text']

    flan_t5_xxl_preds.append(question)

In [ ]:
flanT5_preds=pd.DataFrame([],columns=['answer','title','prediction'])
flanT5_preds['answer']=asks_ds_filtered['validation']['answers.text']
flanT5_preds['title']=asks_ds_filtered['validation']['title']
flanT5_preds['prediction']=flan_t5_xxl_preds
flanT5_preds.to_csv('./results/flanT5_xxl_predictions_updated')

In [ ]:
bertscore_flant5xxl=bertscore.compute(predictions=flanT5_preds['prediction'],
                  references=flanT5_preds['title'],
                  lang='en')

rouge_flant5xxl=rouge.compute(predictions=flanT5_preds['prediction'],
                  references=flanT5_preds['title'])

flan_t5_results={}
flan_t5_results['bertscore']=bertscore_flant5xxl
flan_t5_results['rouge']=rouge_flant5xxl

In [ ]:
with open("./results/flan_t5_xxl_metrics_updated.pkl",'wb') as f:
    pickle.dump(flan_t5_results,f)

## GPT-3

In [ ]:
asks_ds_filtered=load_from_disk('./data/asks_filtered_data')
openai.api_key = getpass('Enter your OpenAI key: ')
GPT_results={}

In [ ]:
model_engine = "text-ada-001"
max_tokens = 50

GPT_results[model_engine]=[]

for answer in tqdm(asks_ds_filtered['validation']['answers.text']):

    prompt = f"generate a question: {answer}"

    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt[:2000],
        max_tokens=max_tokens,
    )

    question = response.choices[0].text.strip()
    GPT_results[model_engine].append(question)

100%|██████████| 2012/2012 [12:07<00:00,  2.77it/s]


In [ ]:
model_engine = "gpt-3.5-turbo"
max_tokens = 50

GPT_results[model_engine]=[]

for answer in tqdm(asks_ds_filtered['validation']['answers.text']):

    prompt = f"generate a question: {answer}"

    response=openai.ChatCompletion.create(
        model=model_engine,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates questions from text."},
            {"role": "user", "content": prompt},
        ])

    question=response['choices'][0]['message']['content']
    
    GPT_results[model_engine].append(question)

100%|██████████| 2012/2012 [55:47<00:00,  1.66s/it]


In [ ]:
df_GPT_results=pd.DataFrame.from_dict(GPT_results)
df_GPT_results['answer']=asks_ds_filtered['validation']['answers.text']
df_GPT_results['title']=asks_ds_filtered['validation']['title']
df_GPT_results.to_csv('./results/GPT_results')

In [ ]:
df_GPT_results=pd.read_csv('./results/GPT_results')

In [ ]:
GPT_results={}

In [ ]:
bertscore_GPT={}
rouge_GPT={}

for key in ["text-ada-001","gpt-3.5-turbo"]:

    bertscore_GPT[key]=bertscore.compute(predictions=df_GPT_results[key],
                  references=df_GPT_results['title'],
                  lang='en')

    rouge_GPT[key]=rouge.compute(predictions=df_GPT_results[key],
                  references=df_GPT_results['title'])

In [ ]:
GPT_metrics={}
GPT_metrics['bertscore']=bertscore_GPT
GPT_metrics['rouge']=rouge_GPT

f=open("./results/GPT_metrics.pkl",'wb')
pickle.dump(GPT_metrics,f)
f.close()

#Metrics

## FLAN-T5-XXL

In [ ]:
with open("./results/flan_t5_xxl_metrics_updated.pkl",'rb') as f:
    flan_t5_results=pickle.load(f)

for key,value in flan_t5_results['bertscore'].items():
    if 'hash' not in key:
        print((key,np.mean(value)))

for key,value in flan_t5_results['rouge'].items():
    print((key,np.mean(value)))

('precision', 0.8809607317800787)
('recall', 0.8610130251638932)
('f1', 0.8706823960983493)
('rouge1', 0.19514456806711303)
('rouge2', 0.054903378286166836)
('rougeL', 0.17377916390269776)
('rougeLsum', 0.17374301359073874)


## GPT-3.5

In [3]:
with open('./results/GPT_metrics.pkl', "rb") as file1:
    GPT_metrics = pickle.load(file1)

In [9]:
GPT_metrics['bertscore']['text-ada-001'].keys()

dict_keys(['precision', 'recall', 'f1', 'hashcode'])

In [13]:
for metric_name in GPT_metrics:
    for model_name in GPT_metrics[metric_name]:
        metric=GPT_metrics[metric_name][model_name]
        print(f'metric: {metric_name}, model: {model_name}')
        for measure,value in metric.items():
            if 'hashcode' in measure:
                continue
            if type(value)==list:
                value=np.mean(value)
            print(f'{measure}: {value}')
        print('\n')

metric: bertscore, model: text-ada-001
precision: 0.8367684350513795
recall: 0.8323285973984489
f1: 0.8342200036551322


metric: bertscore, model: gpt-3.5-turbo
precision: 0.8615672171234373
recall: 0.8672474208574883
f1: 0.8642142171828932


metric: rouge, model: text-ada-001
rouge1: 0.09724690973684282
rouge2: 0.01743578424688018
rougeL: 0.08273397687055292
rougeLsum: 0.08372271009777937


metric: rouge, model: gpt-3.5-turbo
rouge1: 0.17994005901807641
rouge2: 0.03984885743497718
rougeL: 0.14752995001099267
rougeLsum: 0.14760859854733205




## Fine-tuned Models

In [ ]:
fine_tuned_metrics={}

In [ ]:
for model_name in tqdm(model_list_large):

    fine_tuned_metrics[model_name]={}
    
    df_orig = pd.read_csv(f'./results/{model_name}_validation_{max_length}')
    df_aug = pd.read_csv(f'./results/yake_top3_{model_name}_validation_{max_length}')

    fine_tuned_metrics[model_name]['bert_score_orig']=bertscore.compute(
        predictions=df_orig['prediction'],
        references=df_orig['title'],
        lang='en')
    
    fine_tuned_metrics[model_name]['bert_score_aug']=bertscore.compute(
        predictions=df_aug['prediction'],
        references=df_aug['title'],
        lang='en')
    
    fine_tuned_metrics[model_name]['rouge_orig']=rouge.compute(
        predictions=df_orig['prediction'],
        references=df_orig['title'])
    
    fine_tuned_metrics[model_name]['rouge_aug']=rouge.compute(
        predictions=df_aug['prediction'],
        references=df_aug['title'])

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [01:15<00:00, 25.03s/it]


In [ ]:
f=open("./results/fine_tuned_metrics.pkl",'wb')
pickle.dump(fine_tuned_metrics,f)
f.close()

In [17]:
with open("./results/fine_tuned_metrics.pkl",'rb') as f:
    fine_tuned_metrics=pickle.load(f)

In [ ]:
fine_tuned_metrics[model_name].keys()

dict_keys(['bert_score_orig', 'bert_score_aug', 'rouge_orig', 'rouge_aug'])

In [21]:
for model_name in model_list_large:
    print(f'metrics for model: {model_name}')
    print('-'*len(f'metrics for model: {model_name}'))
    metrics=fine_tuned_metrics[model_name]
    for metric in metrics:
        print(f'metric is {metric}')
        for key,val in metrics[metric].items():
            if 'hash' in key:
                continue
            print(f'{key}: {np.mean(val)}')
        print('\n')
    

metrics for model: bart-large
-----------------------------
metric is bert_score_orig
precision: 0.8899752889606161
recall: 0.8729644818583255
f1: 0.8811809643658681


metric is bert_score_aug
precision: 0.888292364287329
recall: 0.8716021830412313
f1: 0.8796704821214524


metric is rouge_orig
rouge1: 0.2275832511870963
rouge2: 0.07480613828332522
rougeL: 0.20097980955640837
rougeLsum: 0.20083733815731775


metric is rouge_aug
rouge1: 0.21808715125561373
rouge2: 0.07056811955686101
rougeL: 0.19303432868912748
rougeLsum: 0.19319750132001112


metrics for model: pegasus-arxiv
--------------------------------
metric is bert_score_orig
precision: 0.8439137447129187
recall: 0.8662594867866509
f1: 0.8547484932493263


metric is bert_score_aug
precision: 0.8429810482986405
recall: 0.8641257763619925
f1: 0.8532407622688096


metric is rouge_orig
rouge1: 0.1565940020206485
rouge2: 0.03857901691443188
rougeL: 0.1339474926496237
rougeLsum: 0.1338670022890464


metric is rouge_aug
rouge1: 0.149301

In [ ]:
fine_tuned_metrics['bart-large']['rouge_orig'].keys()

dict_keys(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])

# Examples

In [ ]:
flan_t5_small_df=pd.read_csv(f'./results/flan-t5-small_validation_50')
flan_t5_base_df=pd.read_csv(f'./results/flan-t5-base_validation_50')

yake_flan_t5_small_df=pd.read_csv(f'./results/yake_top3_flan-t5-small_validation_50')
yake_flan_t5_base_df=pd.read_csv(f'./results/yake_top3_flan-t5-base_validation_50')

In [ ]:
print(flan_t5_small_df[flan_t5_small_df['title'].str.contains('silly putty')]['prediction'])
print(flan_t5_base_df[flan_t5_base_df['title'].str.contains('silly putty')]['prediction'])
print(yake_flan_t5_small_df[yake_flan_t5_small_df['title'].str.contains('silly putty')]['prediction'])
print(yake_flan_t5_base_df[yake_flan_t5_base_df['title'].str.contains('silly putty')]['prediction'])

1384    why is silly putty a fun fluid?
Name: prediction, dtype: object
1384    why is silly putty a fun type of fluid?
Name: prediction, dtype: object
1384    why is silly putty a fun fluid?
Name: prediction, dtype: object
1384    why is silly putty a fun type of fluid?
Name: prediction, dtype: object


In [ ]:
print(flan_t5_small_df[flan_t5_small_df['title'].str.contains('why water?')]['prediction'])
print(flan_t5_base_df[flan_t5_base_df['title'].str.contains('why water?')]['prediction'])
print(yake_flan_t5_small_df[yake_flan_t5_small_df['title'].str.contains('why water?')]['prediction'])
print(yake_flan_t5_base_df[yake_flan_t5_base_df['title'].str.contains('why water?')]['prediction'])

21    why is water so harmful to power plants?
Name: prediction, dtype: object
21    why is water used in power plants?
Name: prediction, dtype: object
21    why is water so harmful to power plants?
Name: prediction, dtype: object
21    why is water used in power plants?
Name: prediction, dtype: object


In [ ]:
df_original.to_csv('./results/sample_original_dataset_50')

In [ ]:
df_augmented=pd.DataFrame([],columns=['answer','title'])
max_length=50

df_augmented['answer']=yake_asks_ds_filtered['validation'][samples]['answers.text']
df_augmented['title']=yake_asks_ds_filtered['validation'][samples]['title']

model_list_large=['bart-large','pegasus-arxiv','flan-t5-base']

for model_name in model_list_large:
    df_file=f'./results/yake_top3_{model_name}_{max_length}'
    df_augmented[model_name]=pd.read_csv(df_file)['prediction']
    df_augmented[model_name]=df_augmented[model_name].map(
        lambda x:eval(x)[0]['generated_text'])

In [ ]:
df_augmented.to_csv('./results/sample_augmented_dataset_50')

In [ ]:
!pip install wandb
import wandb

In [ ]:
with wandb.init(project='Question_Generation', 
                 entity=None, 
                 job_type='training',
                 name='log_samples_original_dataset') as run:
    run.log({'original_dataset_samples':
             wandb.Table(dataframe=df_original)})

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
with wandb.init(project='Question_Generation', 
                 entity=None, 
                 job_type='training',
                 name='log_samples_augmented_dataset') as run:
    run.log({'augmented_dataset_samples':
             wandb.Table(dataframe=df_augmented)})

wandb: Currently logged in as: dmeltzer. Use `wandb login --relogin` to force relogin
